# Astroquerys

- In this notebook I will log into the Gaia archive as a registered ESA user (must be registered to obtain data fr more than 3000 star, if you need less than 3000 then you don't have to register)
- Use a query to load all the necessary data.
- This query will filter the data with any constraints (brighter than M=x, stars with parallax error < x, or in a certain area)
- This data will be saved as a .vot file that is hard to read, so I found it easier to write out a .dat table

## This notebook can be used to request information with querys and save it as a table to upload on future notebooks


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import astropy.units as u 
from astropy.coordinates import SkyCoord
from astropy.units import Quantity #Quantity is a number with an associated unit
import astropy.table
import astropy.io.ascii

In [4]:
from astroquery.gaia import Gaia #might have to install module (using terminal "conda install gaia" "y")
Gaia.login(user='libarr01', password='Llavigne.614') #log into the ESA website

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [5]:
tables = Gaia.load_tables(only_names=True) #use this to see the available tables the gaia module contains
for table in (tables):
    print (table.get_qualified_name())

Retrieving tables...
Parsing tables...
Done.
public.public.dual
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiadr1.allwise_neighbourhood
gaiadr1.gaiadr1.gsc23_best_neighbour
gaiadr1.gaiadr1.gsc23_neighbourhood
gaiadr1.gaiadr1.ppmxl_best_neighbour
gaiadr1.gaiadr1.ppmxl_neighbourhood
gaiadr1.gaiadr1.sdss_dr9_best_neighbour
gaiadr1.gaiadr1.sdss_dr9_neighbourhood
gaiadr1.gaiadr1.tmass_best_neighbour
gaiadr1.gaiadr1.tmass_neighbourhood
gaiadr1.gaiadr1.ucac4_best_neighbour
gaiadr1.gaiadr1.ucac4_neighbourhood
gaiadr1.gaiadr1.urat1_best_neighbour
gaiadr1.gaiadr1.urat1_neighbourhood
gaiadr1.gaiadr1.cepheid

In [7]:
table = Gaia.load_table('gaiadr2.gaia_source') #use this to read a lil bit about the table you are interested in
print (table)

Retrieving table 'gaiadr2.gaia_source'
Parsing table 'gaiadr2.gaia_source'...
Done.
TAP Table name: gaiadr2.gaiadr2.gaia_source
Description: This table has an entry for every Gaia observed source as listed in the
Main Database accumulating catalogue version from which the catalogue
release has been generated. It contains the basic source parameters,
that is only final data (no epoch data) and no spectra (neither final
nor epoch).
Num. columns: 96


#### choose the table you want to use

In [ ]:
#extract desired information using a query. 
#"FROM" lets you choose a table
#"SELEC" lets you request what imformation from each star you want 
#"WHERE" will let you add any constraints, area, > or < than something
#to find out the name of the headers I looked at the first 20 rows of the table I wanted to load (trough the GAIA archive on the ESA website)
job = Gaia.launch_job_async("SELECT parallax,phot_g_mean_flux, ra,dec,phot_g_mean_mag,pmra,pmdec \
FROM gaiadr1.gaiadr1.tgas_source \
WHERE abs(parallax_error/parallax)<0.40" \
, dump_to_file=False) #could dump to file but it creates a .vot file that I dont know how to work with 

In [ ]:
r = job.get_results()

In [ ]:
#plot to see that data looks like expected
plt.figure(figsize=(6,6))
plt.style.use('dark_background')
#plt.subplot(111, projection="aitoff")
plt.scatter(r['ra'],r['dec'],color='aqua',s=0.5, alpha=0.025) #ra and dec for the celestial plane
plt.xlabel('Right Ascension')
plt.ylabel('Declination')
plt.grid(True)
plt.title('RA and Dec')

## Write the data to a table to upload on future notebooks

In [ ]:
#separate into arrays, use.data to get rid of header and other things
parallax = r['parallax'].data
flux = r['phot_g_mean_flux'].data
RA = r['ra'].data
DEC = r['dec'].data
magnitude = r['phot_g_mean_mag'].data 
pmRA =r['pmra'].data
pmDEC = r['pmdec'].data

In [ ]:
table = astropy.table.Table([parallax, RA, DEC, flux, magnitude,pmRA,pmDEC],
        names=['Parallax', 'RA', 'DEC', 'Flux', 'Magnitude','pmRA','pmDEC'])

In [ ]:
astropy.io.ascii.write(table, 'TableFromQueryTGAS.dat',format='fixed_width',delimiter='\t')

In [ ]:
plt.scatter(flux,magnitude)

## Create querys to get more stuff you need and then write them out to a table

- using "2" to not mix lists accidentally

In [6]:
job2 = Gaia.launch_job_async("SELECT parallax,phot_g_mean_flux, ra,dec,phot_g_mean_mag,pmra,pmdec \
FROM gaiadr2.gaia_source \
WHERE phot_g_mean_mag<12" \
, dump_to_file=False)

Launched query: 'SELECT parallax,phot_g_mean_flux, ra,dec,phot_g_mean_mag,pmra,pmdec FROM gaiadr2.gaia_source WHERE phot_g_mean_mag<12'
Retrieving async. results...


Query finished.


In [11]:
r2 = job2.get_results()

In [8]:
parallax2 = r2['parallax'].data
flux2 = r2['phot_g_mean_flux'].data
RA2 = r2['ra'].data
DEC2 = r2['dec'].data
magnitude2 = r2['phot_g_mean_mag'].data 
pmRA2 =r2['pmra']
pmDEC2 = r2['pmdec']

In [17]:
len(magnitude2)

3060477

In [12]:
table2 = astropy.table.Table([parallax2, RA2, DEC2, flux2, magnitude2,pmRA2,pmDEC2],
        names=['Parallax', 'RA', 'DEC', 'Flux', 'Magnitude','pmRA','pmDEC']) #I havent been able to upload this table, not sure if because it is too big or something went wrong, but the TGAS table works..) 

In [13]:
astropy.io.ascii.write(table2, 'ADQLtableDR2.dat',format='fixed_width',delimiter='\t')